<a href="https://colab.research.google.com/github/SydanJainen/4EU-DeepLearningCourse/blob/main/Assignments/AssignementCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignments

### We are given a dataset containing 510 images which are categorized according two main characteristics/properties: “C1” and “C2”. The following inputs are available:
* #### `input image', a 110 × 110 × 3 real-valued tensor. The last dimension denotes the number of input channels; the images belong to different combinations of the two properties C1 and C2. For C1 and C2 we have 5 and 6 possible values, respectively;  among the 30 possible couples (C1,C2), we have images for only 24 of them. We have around 20 samples (little more or less) for each of the available couples;  values in each tensor entry are integers in [0, 255];
* #### C1, string, the value for the C1 property. 5 possible values;
* #### C2, string, the value for the C2 property. 6 possible values.

### Design a deep neural network model to predict the class of an image, given by the couple (C1,C2).

### Provide a sketch of each of the following points, then implement your solution.
1. MODEL: Which architecture do you consider the most appropriate for this task, and why;
2. INPUT:
* After a potential preprocessing step, which is the input of the model, and
how is it represented;
3. OUTPUT: How would you design the output layer and why;
4. LOSS: Which loss function would you use to train your model and why;
5. MODEL CONFIGURATION:
 Model composition (composition of layers, regardless their number,
or their dimension, which can be object of tuning)
6. MODEL EVALUATION: How would you assess (in which setting) the
generalization capabilities of the model on unseen data?


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

## go to your datapath
%cd /content/drive/MyDrive/INSEGNAMENTI/4EU+/

# Here you should see the desired files
%ls

Mounted at /content/drive
/content/drive/MyDrive/INSEGNAMENTI/4EU+
input.pkl


In [14]:
import pickle as pk
# Open the pickle data
with open('input.pkl', 'rb') as f:
  data = pk.load(f)

images = data['imgs']
C1 = data['C1']
C2 = data['C2']
print(f"length image:{len(images)}")
print(f"length C1:{len(C1)}")
print(f"length C2:{len(C2)}")

length image:510
length C1:510
length C2:510


In [5]:
#import
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import transforms


import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import random_split

In [18]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert to PyTorch tensors
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # Normalize to [0, 1]
])

for image in images:
  image = transform(image)

In [19]:
for a in np.unique(C1):
  print(a)

dress
pants
shirt
shoes
shorts


In [20]:
for a in np.unique(C2):
  print(a)

black
blue
brown
green
red
white


In [21]:
main_dataset = zip(images, C1, C2)

label_combinations = {}
for i, (c1, c2) in enumerate(zip(C1, C2)):
    label_combinations[i] = f"{c1} {c2}"

unique_labels = set(label_combinations.values())
label_to_index = {label: i for i, label in enumerate(unique_labels)}
print(label_to_index)

enumerated_dataset = []
for i, (image, c1, c2) in enumerate(zip(images, C1, C2)):
  label_combination = f"{c1} {c2}"
  label_index = label_to_index[label_combination]
  enumerated_dataset.append((image, label_index))


{'shorts blue': 0, 'pants white': 1, 'shoes red': 2, 'shoes black': 3, 'shoes brown': 4, 'shoes blue': 5, 'shoes white': 6, 'shirt green': 7, 'shorts black': 8, 'pants brown': 9, 'pants red': 10, 'shoes green': 11, 'shorts white': 12, 'pants blue': 13, 'shirt black': 14, 'shorts green': 15, 'pants black': 16, 'dress white': 17, 'dress blue': 18, 'shirt blue': 19, 'pants green': 20, 'dress black': 21, 'dress red': 22, 'shorts brown': 23}


In [22]:
BATCH_SIZE = 20

train_size = int(0.6 * len(enumerated_dataset))
val_size = int(0.2 * len(enumerated_dataset))
test_size = len(enumerated_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(enumerated_dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def dimension(N,P,S,F) :
  return (int((N + (2*P) - F)/ S) + 1)

In [ ]:
print(dimension)

In [29]:
# model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=32,
                               kernel_size = 3,
                               stride=1,
                               padding=0)
        # dim (28-3)/1 +1 = 26
        #  feature map dim 26x26x32
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        # dim 24 x 24 x 64
        self.maxpool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        # dim 12x12 x 64, 'same' = 9216
        self.dropout1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(9216, 128)
        self.dropout2 = nn.Dropout(0.25)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.maxpool2d(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

#Creating the model
model = CNN()
print(model)

# calling the method on one batch
it = iter(train_loader)
X_batch, Y_batch = next(it)


# Computing the accuracy of a test set
def evaluate(model, test_loader, error):
#model = mlpinplace=False
    correct = 0
    cur_loss = 0;

    for test_imgs, test_labels in test_loader:
        test_imgs, test_labels = test_imgs.cuda(), test_labels.cuda()
        output = model(test_imgs)
        loss = error(output, test_labels)
        cur_loss+=loss.item()
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    return cur_loss/len(test_loader.dataset), correct/len(test_loader.dataset)



#training the model
def fit(model, train_loader, error, test_loader, epochs):
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)


    model.train()
    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        cur_loss = 0;
        correct = 0
        for X_batch, y_batch in train_loader:
            # moving data to GPU
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            optimizer.zero_grad()
            output = model(X_batch)
            loss = error(output, y_batch)
            cur_loss+= loss.item()
            loss.backward()
            optimizer.step()

        train_losses.append(cur_loss/len(train_loader.dataset))

        test_loss, _ = evaluate(model, test_loader, error)
        test_losses.append(test_loss)
        print(f'Epoch : {epoch},  train loss:{train_losses[-1]}, test loss:{test_losses[-1]}')

    return train_losses, test_losses

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxpool2d): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (dropout2): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
<class 'torch.Tensor'>


In [27]:
# train
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
print(dev)
device = torch.device(dev)
model.cuda()

epochs = 15
error = nn.CrossEntropyLoss()
train_losses, test_losses = fit(model,train_loader, error, test_loader, epochs)

plt.plot(train_losses,label="training loss")
plt.plot(test_losses,label="testing loss")
plt.legend(fontsize=15)
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Loss', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.figure()

cuda:0


RuntimeError: Input type (unsigned char) and bias type (float) should be the same

In [ ]:
# validate and test

loss, acc = evaluate(model, train_loader, error)
print(f"Train Accuracy:{acc}, Train loss:{loss}")

loss, acc = evaluate(model, test_loader, error)
print(f"Test Accuracy:{acc}, Test loss:{loss}")

loss, acc = evaluate(model, val_loader, error)
print(f"Validation Accuracy:{acc}, Validation loss:{loss}")